In [11]:
import pandas as pd
import numpy as np
from pycaret.regression import *

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

In [12]:
df = pd.read_csv('encoded_df.csv')

In [13]:
df1 = df.copy()

In [14]:
df1.head()

,Wk,Winner,Pts,Loser,Pts.1,winners,losers,home_rank,away_rank,home_team_code,away_team_code,home_results,win_rank,lose_rank,day_code,hours,null,half,quart,three_quart
0,1,Florida Atlantic,43,Charlotte,13,Florida Atlantic,Charlotte,99,99,32,37,0,99,99,5,19,1,0,0,0
1,1,Florida State,47,Duquesne,7,Florida State,Duquesne,99,99,34,49,0,99,99,5,17,1,0,0,0
2,1,Illinois,38,Wyoming,6,Illinois,Wyoming,99,99,42,224,0,99,99,5,16,1,0,0,0
3,1,Nevada,23,New Mexico State,12,Nevada,New Mexico State,99,99,76,122,1,99,99,5,22,1,0,0,0
4,1,Nevada-Las Vegas,52,Idaho State,21,Nevada-Las Vegas,Idaho State,99,99,74,77,0,99,99,5,15,0,1,0,0


In [15]:
df.shape

(779, 20)

In [16]:
df1.iloc[714:715]

,Wk,Winner,Pts,Loser,Pts.1,winners,losers,home_rank,away_rank,home_team_code,away_team_code,home_results,win_rank,lose_rank,day_code,hours,null,half,quart,three_quart
714,13,Bowling Green,42,Toledo,35,Bowling Green,Toledo,99,99,114,23,1,99,99,1,19,1,0,0,0


In [17]:
train = df1.iloc[:714]
test = df1.iloc[714:]

In [18]:
X_train = train.drop(columns=['Winner', 'Loser', 'winners', 'losers',
                              'null', 'half', 'quart', 'three_quart', 
                              'Pts.1', 'hours'])

X_test = test.drop(columns=['Winner', 'Loser', 'winners', 'losers',
                              'null', 'half', 'quart', 'three_quart', 
                              'Pts.1', 'hours'])

In [19]:
y_train = train['Pts.1']
y_test = test['Pts.1']

In [20]:
X_train

,Wk,Pts,home_rank,away_rank,home_team_code,away_team_code,home_results,win_rank,lose_rank,day_code
0,1,43,99,99,32,37,0,99,99,5
1,1,47,99,99,34,49,0,99,99,5
2,1,38,99,99,42,224,0,99,99,5
3,1,23,99,99,76,122,1,99,99,5
4,1,52,99,99,74,77,0,99,99,5
...,...,...,...,...,...,...,...,...,...,...
709,12,37,6,24,88,213,1,24,6,5
710,12,28,99,99,126,11,0,99,99,5
711,12,23,99,99,127,145,0,99,99,5
712,12,45,99,99,128,155,0,99,99,5


In [ ]:
# Ignoring features with high null values 

demo = setup(data = train, target = 'Pts', 
                   transformation= True, transformation_method = 'yeo-johnson', 
                   transform_target = True, remove_outliers= True,
                   remove_multicollinearity = True, imputation_type='iterative',
                   ignore_low_variance = True, combine_rare_levels = True, fold_shuffle=True,) 



In [ ]:
best = compare_models()

In [21]:
from sklearn.linear_model import Ridge

In [22]:
clf = Ridge(alpha=.001)
clf.fit(X_train, y_train)

Ridge(alpha=0.001)

In [23]:
test_pred = clf.predict(X_test)

print('RMSE: '+str(np.sqrt(mean_squared_error(y_test, test_pred))))
print('R Squared: '+str(r2_score(y_test, test_pred)))
print('MSE TEST: '+str(mean_squared_error(y_test, test_pred)))
print('MAE TEST: '+str(mean_absolute_error(y_test, test_pred)))
print('MAPE: '+str(mean_absolute_percentage_error(y_test, test_pred)))

RMSE: 10.037471481029478
R Squared: 0.09066925180996077
MSE TEST: 100.75083373248009
MAE TEST: 8.273833228112974
MAPE: 3983741310533154.5


In [24]:
# RMSE: 12.98527052295243
# R Squared: 0.003458085844145864
# MSE TEST: 168.61725055425725
# MAE TEST: 10.68224945264794
# MAPE: 0.3613084327156201

# alpha=0 MAPE: 0.36128976519731065

In [25]:
data = {'y_test': y_test, 'y_pred': test_pred}
new_df = pd.DataFrame(data)
diff = (new_df['y_test'] - new_df['y_pred']).abs()
new_df['diff'] = diff
new_df = new_df.sort_values(by='diff', ascending=True)
new_df

,y_test,y_pred,diff
777,20,19.614678,0.385322
725,27,27.546482,0.546482
749,21,21.661597,0.661597
761,20,19.328493,0.671507
726,17,18.812803,1.812803
734,17,18.815893,1.815893
728,17,18.971457,1.971457
727,17,19.021388,2.021388
717,23,20.897485,2.102515
716,24,21.586101,2.413899


In [26]:
new_df['diff'].mean()

8.273833228112975

- michigan awaycode = 110 rank3 WIN
- ohio state homecode = 83 rank2
- 1


- oregon awaycode = 148 rank 10 WIN
- oregon state homecode = 89 rnak 22
- 1


- notre dame awaycode = 142 rank 13
- usc homecode = 100 rank 5 WIN
-  0


- tulene awaycode = 200 rank 19 WIN
- cincinati  homecode = 22 rank 21
- 1


- baylor awaycode = 99 rank 99
- texas homecode=189 rank 24 WIN
- 0


- ms state awaycode=115 rank99
- ole miss homecode=67 rank 20 WIN
- 0


In [33]:
oh = [[14, 31, 2, 3, 83, 110, 1, 3, 2, 5]]
osu = pd.DataFrame(oh, columns=['Wk', 'Pts', 'home_rank', 'away_rank',
                                'home_team_code', 'away_team_code', 
                                'home_results', 'win_rank', 
                                'lose_rank', 'day_code'])
osu

# og = [[14, 31, 22, 10, 89, 148, 1, 10, 22, 5]]
# uo = pd.DataFrame(og, columns=['Wk', 'Pts', 'home_rank', 'away_rank',
#                                 'home_team_code', 'away_team_code', 
#                                 'home_results', 'win_rank', 
#                                 'lose_rank', 'day_code'])
# uo


# sc = [[14, 33, 5, 13, 100, 142, 0, 5, 13, 5]]
# usc = pd.DataFrame(sc, columns=['Wk', 'Pts', 'home_rank', 'away_rank',
#                                 'home_team_code', 'away_team_code', 
#                                 'home_results', 'win_rank', 
#                                 'lose_rank', 'day_code'])
# usc

# tu = [[14, 29, 21, 19, 22, 200, 1, 19, 21, 5]]
# tul = pd.DataFrame(tu, columns=['Wk', 'Pts', 'home_rank', 'away_rank',
#                                 'home_team_code', 'away_team_code', 
#                                 'home_results', 'win_rank', 
#                                 'lose_rank', 'day_code'])
# tul


,Wk,Pts,home_rank,away_rank,home_team_code,away_team_code,home_results,win_rank,lose_rank,day_code
0,14,31,2,3,83,110,1,3,2,5


In [34]:
clf.predict(osu)


array([26.31609043])

In [ ]:
# ohio state 30.68
# michigan 26.316

# Oregon 30.783
# Oregon State 24.936

# USC 32.981
# Notre Dame 23.154

# tulane 29.4244
# Cinncinati 24.165